# Fama-French 3-Factor Model
Welcome to this notebook dedicated to exploring the Fama-French 3-Factor Model and its application in predicting expected returns. In this tutorial, we will dive into the world of asset pricing models and learn how to estimate the parameters of the Fama-French 3-Factor Model using Python. We will then leverage these estimated parameters to make predictions of the expected return for a specific firm.

The Fama-French 3-Factor Model, developed by Eugene Fama and Kenneth French, is a widely used asset pricing model in finance. It extends the traditional Capital Asset Pricing Model (CAPM) by incorporating additional factors that capture market anomalies and asset characteristics.

The three factors in the model are:

1. Market Risk Premium: The excess return of the overall market compared to the risk-free rate.
2. Size Factor: The excess return of small-cap stocks compared to large-cap stocks.
3. Value Factor: The excess return of value stocks compared to growth stocks.

By considering these three factors, the Fama-French 3-Factor Model aims to provide a more comprehensive explanation of asset returns. It enables us to assess whether an asset's return can be attributed to systematic risk, company size, or the value characteristics of the asset.

In this notebook, we will walk through the steps of estimating the parameters of the Fama-French 3-Factor Model. We will start by gathering historical market and firm-specific data and constructing the necessary factor portfolios. Next, we will perform a regression analysis using Python to estimate the model parameters. Finally, we will utilize these estimated parameters to predict the expected return for a specific firm based on its exposure to the three factors.

In [2]:
import pandas as pd
import yfinance as yf
import numpy as np
from datetime import datetime
from sklearn.linear_model import LinearRegression

fama_french = pd.read_csv('Data/fama_french.csv', sep=',').dropna()

fama_french.dropna(inplace=True)
fama_french.loc[:, 'date'] = fama_french['index'].apply(lambda x: pd.to_datetime(x))
fama_french.drop(['index'], axis=1, inplace=True)
fama_french.set_index('date', inplace=True, drop=True)
fama_french

,Mkt-RF,SMB,HML,RF
date,,,,
1926-07-01,0.10,-0.25,-0.27,0.009
1926-07-02,0.45,-0.33,-0.06,0.009
1926-07-06,0.17,0.30,-0.39,0.009
1926-07-07,0.09,-0.58,0.02,0.009
1926-07-08,0.21,-0.38,0.19,0.009
...,...,...,...,...
2023-03-27,0.27,0.23,1.02,0.016
2023-03-28,-0.17,-0.17,0.74,0.016
2023-03-29,1.39,-0.28,-0.50,0.016


##### Ford

In [3]:
ford = yf.download('F','2018-01-01',datetime.today(), progress=False)['Adj Close'].pct_change()

In [4]:
data = pd.concat([fama_french, ford.rename('ford')], axis=1).dropna()
data

,Mkt-RF,SMB,HML,RF,ford
2018-01-03,0.59,-0.39,-0.19,0.006,0.007899
2018-01-04,0.42,-0.26,0.25,0.006,0.017241
2018-01-05,0.66,-0.35,-0.25,0.006,0.016949
2018-01-08,0.19,-0.15,0.04,0.006,-0.003788
2018-01-09,0.15,-0.35,-0.04,0.006,-0.005323
...,...,...,...,...,...
2023-03-27,0.27,0.23,1.02,0.016,0.000869
2023-03-28,-0.17,-0.17,0.74,0.016,0.006944
2023-03-29,1.39,-0.28,-0.50,0.016,0.038793
2023-03-30,0.51,-0.55,-0.59,0.016,0.019917


### Parameter estimation
The parameters of the fama-French 3-Factor model can easily be estimated using a linear regression model:

In [8]:
regression = LinearRegression(fit_intercept=True)
y = data['ford'] - data['RF']
X = data[['Mkt-RF', 'SMB', 'HML']]
regression.fit(X, y)

LinearRegression()

In [9]:
# Coefficients:
for i, col in enumerate(['Mkt-RF', 'SMB', 'HML']):
    print('coefficient {c}: {n}'.format(c=col, n=round(regression.coef_[i], 4)))
print('Intercept: ', round(regression.intercept_, 4))

coefficient Mkt-RF: 0.0115
coefficient SMB: 0.0059
coefficient HML: 0.0069
Intercept:  -0.005


## Expected return

In [11]:
m = 2
smb = 0.5
hml = 0.01
expected_return = -0.005 + 0.0115*m + 0.0059*smb + 0.0069*hml
print(expected_return + 0.016)

0.037018999999999996
